### Import the dependencies

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
df = pd.read_csv('creditcard.csv')

In [5]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [9]:
#check no of missing values in each colomn
df.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [10]:
# distribution of legit and fraudulent transection
df['Class'].value_counts()

Class
0    284315
1       492
Name: count, dtype: int64

### This data is highly unblanced we'll apply some techniques to balance it
#### 0 = normal transection
#### 1 = fruad transection

In [11]:
# separate the data frame
legit = df[df.Class == 0]
fruad = df[df.Class == 1]

In [14]:
print(legit.shape)
print(fruad.shape)

(284315, 31)
(492, 31)


In [15]:
# satistical measure of data
legit.Amount.describe()

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [18]:
fruad.Amount.describe()

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

In [20]:
# compare the values for both transactions
df.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,...,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


### Under-sampling 
#### Build a sample dataset having similar distribution of normal and fruad transections 
#### no of fruad transection = 492

In [21]:
legit_sample = legit.sample(n=492)

#### concatenating the 2 data frames i.e legit and fruad

In [22]:
new_df = pd.concat([legit_sample,fruad], axis = 0)

In [23]:
new_df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
107920,70677.0,1.165994,-0.121103,0.050557,0.081347,-0.552503,-1.106659,0.137425,-0.137194,-0.097538,...,-0.005111,-0.190127,-0.042334,0.574615,0.313183,0.978464,-0.121253,-0.006966,49.95,0
283497,171637.0,1.956745,-0.425646,-0.422829,0.270620,-0.553728,-0.225994,-0.607261,0.124632,1.166305,...,-0.159638,-0.391643,0.316509,-0.472706,-0.435573,-0.615486,0.024583,-0.045463,12.99,0
154466,101673.0,-1.920193,-1.227348,0.976062,1.638566,-1.851759,1.991724,3.397067,-0.218861,1.345847,...,-0.307644,-1.658620,1.773972,0.404171,0.643117,-0.899205,-0.127632,-0.024613,823.89,0
106548,70006.0,-1.750726,1.187191,0.733108,-0.172249,0.278609,0.146313,0.224221,0.463502,-0.033465,...,-0.282991,-0.331406,0.256190,-0.356356,-0.313042,0.017222,-0.673486,-0.722859,1.98,0
49957,44278.0,-0.638107,0.063446,1.715773,-1.160283,0.595187,1.535115,0.036465,0.321507,-1.684216,...,-0.201034,0.141186,-0.019906,-1.017795,-0.648668,0.469331,-0.136856,-0.129312,10.00,0


In [29]:
new_df['Class'].value_counts()

Class
0    492
1    492
Name: count, dtype: int64

In [30]:
new_df.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,91610.839431,0.063847,0.027732,0.027579,0.028331,-0.116162,0.029801,0.051044,0.038139,0.074463,...,0.021157,-0.042819,-0.034104,0.038653,0.001172,0.041101,0.011449,-0.005630,-0.006159,95.378598
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


#### splitting the data into targets and features

In [32]:
X = new_df.drop(columns ='Class',axis=1)
Y = new_df['Class']

In [36]:
print(X)
print(Y)

107920    0
283497    0
154466    0
106548    0
49957     0
         ..
279863    1
280143    1
280149    1
281144    1
281674    1
Name: Class, Length: 984, dtype: int64


#### spliting the datinto training and testing data

In [38]:
X_train, X_test, y_train, y_test = train_test_split (X, Y, test_size=0.22, stratify=Y, random_state=5)

In [39]:
print(X.shape, X_train.shape, X_test.shape)

(984, 30) (767, 30) (217, 30)


### Model traing 
#### Logistic regression

In [40]:
model = LogisticRegression()

In [41]:
# training our model 
model.fit(X_train,y_train)

C:\Users\mh407\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

### Model evaluation

In [43]:
X_train_pred =model.predict(X_train) 
training_data_accuracy = accuracy_score(X_train_pred,y_train)

In [49]:
print("the accuracy of training data :", training_data_accuracy)

the accuracy of training data : 0.9361147327249022


In [47]:
# accuracy on test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, y_test)

In [48]:
print("the accuracy of testing data :", test_data_accuracy)

the accuracy of testing data : 0.9493087557603687
